In [1]:
import cv2
import numpy as np
import opencv_jupyter_ui as jcv2
from feat import Detector
from IPython.display import Image
from feat.utils import FEAT_EMOTION_COLUMNS
import time as t
from time import sleep
from furhat_remote_api import FurhatRemoteAPI

import time
import sys
import trace
from pathlib import Path
import multiprocessing
import threading
import os
from torch import nn
import torch
import ray

### Furhat setup

In [2]:
FURHAT_IP = "127.0.1.1"

furhat = FurhatRemoteAPI(FURHAT_IP)


In [5]:
furhat.set_led(red=100, green=50, blue=50)

{'message': 'Successfully changed my LEDs', 'success': True}

In [6]:
furhat.say(text="Would you like to take a break?", blocking=True)
result = furhat.listen()
texts = result.__dict__['_message'].split(' ')
print(texts)
if "yes" in texts:
    furhat.say(text="Great! let's take a 10 second break", blocking=True)
    furhat.gesture(name="CloseEyes")
    sleep(10)
    furhat.gesture(name="OpenEyes")
    furhat.say(text="Let's continue.")
      # Exit the loop if the user wants a break

elif "no" in texts:
    furhat.say(text="Great! Let's continue.")
      # Exit the loop if the user doesn't want a break

else:
    furhat.say(text="I'm sorry, I didn't get that, can you repeat it?")

['no']


### Detector setup

In [7]:
detector = Detector(device="cpu")

In [8]:
class MLP(nn.Module):
    def __init__(self, features_in=2, features_out=3):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(features_in, 10),
            nn.ReLU(),
            nn.Linear(10, features_out),
        )

    def forward(self, input):
        return self.net(input)
    
path=Path(os.getcwd()).parent
DIR_PATH=str(path) + '\\'

device = "cpu" #"cuda" if torch.cuda.is_available() else
model = torch.load(Path(DIR_PATH + 'models\\best_model_12.pt') ).to(device) #best_emo_model(local_path).to(device)

In [9]:
expression = {"anger": 6, "disgust": 5 , "fear": 4, "happiness": 1, "neutral": 0, "sadness": 2, "surprise": 3}
def return_emo(loc):
    for key,_ in expression.items():
        if expression[key] == loc:
            return key

In [10]:
EMO_LIST= []

In [11]:
#@ray.remote
def capture_and_return_emos(t):
    fl = False
    start= time.time()
    cap = cv2.VideoCapture(1)
    print("Capture on")
    flag=True
    while flag: #or not event.is_set()
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to capture the frame.")
            break

        detected_faces = detector.detect_faces(frame)
        detected_landmarks = detector.detect_landmarks(frame, detected_faces)
        detected_aus = detector.detect_aus(frame, detected_landmarks)

        for faces,au_units in zip(detected_faces,detected_aus): #access only one frame
            for i in range(len(faces)): #access all faces detected in the frame
                au_arr=model(torch.tensor(au_units[i]).to(device)).cpu()
                max_loc=np.argmax(au_arr.softmax(dim=0).numpy())
                emotion=return_emo(max_loc)
                EMO_LIST.append(emotion)
                x, y, w, h, p = faces[i]
                # Drawing a rectangle around the detected face
                cv2.rectangle(frame, (int(x), int(y)), (int(w), int(h)), (0,0 , 255), 2)

                # Displaying the emotion label on top of the rectangle
                cv2.putText(frame, emotion, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                stop = time.time()
                # if event.is_set():
                #     flag=False
                #     break
                if stop - start >= t:
                    print(fl)
                    fl = True
                    flag=False
                    break                                   
                 #key == 27:

    # jcv2.destroyAllWindows()
    cap.release()
    print("Capture off")

In [12]:
# event = threading.Event()
# stop_threads = False
# t1 = thread_with_trace(target = capture_and_return_emos(event,)) #lambda : stop_threads,

# t1.start()
# time.sleep(5)
# event.set()
# t1.kill()
# # start = time.time()
# # stop = time.time()
# t1.join()
# # if stop - start >=7:
#     # stop_threads = True
# fl=False

# process1 = multiprocessing.Process(target=capture_and_return_emos(lambda: fl,))
# process1.start()
# start= time.time()
# time.sleep(7)
# stop = time.time()
# if stop - start >= 7:
#     fl = True
# else:
#     fl= False
    
# process1.terminate()
# process1.kill()

In [15]:
from time import sleep

# @ray.remote
def bsay(line):
    furhat.say(text=line, blocking=True)
    sleep(3)
    furhat.say(text=line, blocking=True)
    sleep(3)



# # Execute func1 and func2 in parallel.
# ray.get([bsay.remote("Hello everybody!!"), capture_and_return_emos.remote(6)])
queue = multiprocessing.Queue()
# process1 = multiprocessing.Process(target=bsay, args=("Hello everybody!!",))
process2 = multiprocessing.Process(target=capture_and_return_emos(6)) #, daemon=False
# process1.start()
process2.start()
bsay("Hello everybody!!")

# process1.join()

# process1.terminate()
# process2.terminate()



# p1 = threading.Thread(target=capture_and_return_emos(6))
# p1.start()
# p2 = threading.Thread(target=bsay("Hello everybody!!"))
# p2.start()
# p2.join()



Capture on


c:\Users\Audit\anaconda3\envs\iis\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\Audit\anaconda3\envs\iis\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to

False
Capture off


In [11]:
process1.start()
process2.start()


process1.join()
process2.join()

In [10]:
#len(EMO_LIST)
max=0
for i in np.unique(EMO_LIST):
    print(i, np.char.count(EMO_LIST, i).sum())
    if np.char.count(EMO_LIST, i).sum() > max:
        max = np.char.count(EMO_LIST, i).sum()
        emo = i
print(emo, max)

neutral 4
neutral 4


In [39]:
EMO_LIST

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'anger',
 'happiness',
 'happiness',
 'happiness',
 'anger',
 'happiness']

In [13]:
st = time.time()

for i in range(10000):
    print('', end='')
    ft=time.time()

print('\n', f'{(ft-st):.2f}')



 0.02


In [ ]:
def task1():
    print("Task 1 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 1: {}".format(os.getpid()))
 
def task2():
    print("Task 2 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 2: {}".format(os.getpid()))

In [ ]:
print("ID of process running main program: {}".format(os.getpid()))

print("Main thread name: {}".format(threading.current_thread().name))

t1 = threading.Thread(target=task1, name='t1')
t2 = threading.Thread(target=task2, name='t2')

t1.start()
t2.start()

t1.join()
t2.join()